# Molecular Hydrogen (H2) Exemplar
## Step 0: Import various libraries

In [1]:
# Imports for QSCOUT
import jaqalpaq
from jaqalpaq.core import circuitbuilder
from jaqalpaq.core.circuit import normalize_native_gates
from jaqalpaq import emulator
from qscout.v1 import native_gates
from qscout.v1 import noisy

# Imports for basic mathematical functionality
from math import pi
import numpy as np

# Imports for OpenFermion(-PySCF)
import openfermion as of
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf

# Import for VQE optimizer
from scipy import optimize

## Step 1: SCF calculation to assemble the second-quantized Hamiltonian

In [2]:
# Set the basis set, spin, and charge of the H2 molecule
basis = 'sto-3g'
multiplicity = 1
charge = 0

# Set calculation parameters
run_scf = 1
run_fci = 1
delete_input = True
delete_output = False

# Generate molecule at some bond length (0.7474 Angstroms here)
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., 0.7474))]
molecule = MolecularData(geometry, basis, multiplicity, charge, filename='./H2-singlet-0.7474') #Set file location of data

# Run pyscf to generate new molecular data for sto-3g H2
molecule = run_pyscf(molecule, run_scf=run_scf, run_fci=run_fci, verbose=False)

print("Bond Length in Angstroms: {}".format(0.7474))
print("FCI (Exact) energy in Hartrees: {}".format(molecule.fci_energy))

Bond Length in Angstroms: 0.7474
FCI (Exact) energy in Hartrees: -1.137175710240685


## Step 2: Convert the fermionic Hamiltonian to a qubit Hamiltonian

In [3]:
# Get the fermionic Hamiltonian for H2 and map it using the BK encoding
hamiltonian = molecule.get_molecular_hamiltonian()
hamiltonian_ferm = of.get_fermion_operator(hamiltonian)
hamiltonian_bk = of.bravyi_kitaev(hamiltonian_ferm)

# Define Pauli strings that appear in the reduced two-qubit Hamiltonian
#          q0 ,  q1
terms = [[None, None], ['Z', None], [None, 'Z'], ['Z', 'Z'], ['X', 'X'], ['Y', 'Y']]

# Calculate effective coefficients for the reduced two-qubit Hamiltonian
# Derivation follows arXiv:1803.10238v2 appendix A-2
fs = hamiltonian_bk.terms #Old coefficients from OpenFermion Hamiltonian
c0 = (fs[()] + fs[(1, 'Z'),] + fs[(1, 'Z'), (3, 'Z'),]).real
c1 = (fs[(0, 'Z'),] + fs[(0, 'Z'), (1, 'Z'),]).real
c2 = (fs[(2, 'Z'),] + fs[(1, 'Z'), (2, 'Z'), (3, 'Z'),]).real
c3 = (fs[(0, 'Z'), (2, 'Z'),] + fs[(0, 'Z'), (1, 'Z'), (2, 'Z'),] + fs[(0, 'Z'), (2, 'Z'), (3, 'Z'),] + fs[(0, 'Z'), (1, 'Z'), (2, 'Z'), (3, 'Z')]).real
c4 = (fs[(0,'X'), (1, 'Z'), (2, 'X'),] + fs[(0, 'X'), (1, 'Z'), (2, 'X'), (3, 'Z'),]).real
c5 = (fs[(0, 'Y'), (1, 'Z'), (2, 'Y'),] + fs[(0, 'Y'), (1, 'Z'), (2, 'Y'), (3, 'Z'),]).real
cs = [c0, c1, c2, c3, c4, c5] #New coefficients are linear combinations of old coefficients

## Step 3: Define UCC Ansatz circuit in JaqalPaq

In [4]:
def ansatz(theta, term):
    builder = circuitbuilder.CircuitBuilder(native_gates=normalize_native_gates(native_gates.NATIVE_GATES))

    # Create a qubit register
    q = builder.register('q', 2)

    # Prepare the Hartree Fock state
    builder.gate('prepare_all')
    builder.gate('Px', q[0])
    
    # Apply the UCC Ansatz exp[-i*theta(X1 Y0)]
    builder.gate('MS', q[1], q[0], 0, np.pi/2)
    builder.gate('Rz', q[1], theta)
    builder.gate('MS', q[1], q[0], 0, -np.pi/2)

    # Change basis for measurement depending on term
    for j, qubit in enumerate(term):
        if qubit == 'X':
            builder.gate('Sy', ('array_item', q, j))
            builder.gate('Px', ('array_item', q, j))
        if qubit == 'Y':
            builder.gate('Sxd', ('array_item', q, j))

    builder.gate('measure_all')

    circuit = builder.build()

    return circuit

## Step 4: Define functions to calculate energy expectation value of Ansatz state

In [5]:
def ansatz_sampling(theta, sample_noise):
    term_probs = []
    for i in range(len(terms)):
        if sample_noise:
            probs = np.zeros(4)
            circuit = ansatz(theta, terms[i])
            sim_result = emulator.run_jaqal_circuit(circuit) #Run circuit on emulator
            sim_probs = sim_result.subcircuits[0].probability_by_int
            sample = np.random.choice(4, size=n_samples, p=sim_probs) #Sample from distribution to get probabilities
            
            for count in sample:
                probs[count] += 1 #Increment state counter
            probs = probs/n_samples #Determine probabilities from sampling
            term_probs += [probs] #Combine lists of probs of each term in Hamiltonian
                
        else: #Exact solution without sampling
            circuit = ansatz(theta, terms[i])
            sim_result = emulator.run_jaqal_circuit(circuit) #Run circuit on emulator
            sim_probs = sim_result.subcircuits[0].probability_by_int
            term_probs += [sim_probs]
    
    return term_probs

In [6]:
# Calculate energy of one term of the Hamiltonian for one possible state
def term_energy(term, state, coefficient, prob):
    parity = 1
    for i in range(len(term)):
        #Change parity if state is occupied and is acted on by a pauli operator
        if term[i] != None and state[i] == '1':
            parity = -1*parity
    return coefficient*prob*parity

# Calculate energy of the molecule for a given value of theta
def calculate_energy(theta, sample_noise):
    energy = 0
    probs = ansatz_sampling(theta[0], sample_noise) #Convert tuple (from optimization) to float for circuit
    for i in range(len(terms)): #For each term in the hamiltonian
        for j in range(len(probs[0])): #For each possible state
            term = terms[i]
            state = '{0:02b}'.format(j)[::-1] #convert state to binary (# of qubits)
            coefficient = cs[i].real
            prob = probs[i][j]
            #print(term, state, coefficient, prob)
            energy += term_energy(term, state, coefficient, prob)
    return energy

## Step 5: Minimize the energy expectation value in  𝜃

In [8]:
# Minimize the energy using classical optimization
n_samples = 10000
optimize.minimize(fun=calculate_energy, x0=[0.01], args=([True]), method="COBYLA")

     fun: -1.1372235230213579
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 20
  status: 1
 success: True
       x: array([-0.255525])

## Step 6: Loop over previous steps to calculate ground state energy at different bond lengths

In [9]:
# Set the basis set, spin, and charge of the H2 molecule
basis = 'sto-3g'
multiplicity = 1
charge = 0

# Set calculation parameters
run_scf = 1
run_fci = 1
delete_input = True
delete_output = False

optimized_energies = [[], []]
exact_energies = []

# Loop over bond lengths from 0.3 to 1.3 angstroms
n_samples = 10000 # Sample circuit
n_pts = 11 # Number of points
bond_lengths = np.linspace(0.3,1.3,n_pts)
for diatomic_bond_length in bond_lengths:
    # Generate molecule at some bond length
    geometry = [('H', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
    description=str(round(diatomic_bond_length, 2))
    molecule = MolecularData(
        geometry, basis, multiplicity, charge,
        description=description,
        filename='./H2_single_dissociation_'+description)

    # Run pyscf to generate new molecular data for sto-3g H2
    molecule = run_pyscf(molecule, run_scf=run_scf, run_fci=run_fci, verbose=False)
    
    # Get the fermionic Hamiltonian for H2 and map it into qubits using the BK encoding
    hamiltonian = molecule.get_molecular_hamiltonian()
    hamiltonian_ferm = of.get_fermion_operator(hamiltonian)
    hamiltonian_bk = of.bravyi_kitaev(hamiltonian_ferm)

    # Define Pauli strings that appear in the reduced two-qubit Hamiltonian
    #          q0 ,  q1
    terms = [[None, None], ['Z', None], [None, 'Z'], ['Z', 'Z'], ['X', 'X'], ['Y', 'Y']]

    # Calculate effective coefficients for the reduced two-qubit Hamiltonian
    # Derivation follows arXiv:1803.10238v2 appendix A-2
    fs = hamiltonian_bk.terms #Old coefficients from OpenFermion Hamiltonian
    c0 = (fs[()] + fs[(1, 'Z'),] + fs[(1, 'Z'), (3, 'Z'),]).real
    c1 = (fs[(0, 'Z'),] + fs[(0, 'Z'), (1, 'Z'),]).real
    c2 = (fs[(2, 'Z'),] + fs[(1, 'Z'), (2, 'Z'), (3, 'Z'),]).real
    c3 = (fs[(0, 'Z'), (2, 'Z'),] + fs[(0, 'Z'), (1, 'Z'), (2, 'Z'),] + fs[(0, 'Z'), (2, 'Z'), (3, 'Z'),] + fs[(0, 'Z'), (1, 'Z'), (2, 'Z'), (3, 'Z')]).real
    c4 = (fs[(0,'X'), (1, 'Z'), (2, 'X'),] + fs[(0, 'X'), (1, 'Z'), (2, 'X'), (3, 'Z'),]).real
    c5 = (fs[(0, 'Y'), (1, 'Z'), (2, 'Y'),] + fs[(0, 'Y'), (1, 'Z'), (2, 'Y'), (3, 'Z'),]).real
    cs = [c0, c1, c2, c3, c4, c5] #New coefficients are linear combinations of old coefficients

    # Minimize the expectation value of the energy using a classical optimizer
    exact_energies.append(molecule.fci_energy)
    print("R={}\t Exact Energy: {}".format(str(round(diatomic_bond_length, 2)), molecule.fci_energy))
    
    for i in range(2):
        result = optimize.minimize(fun=calculate_energy, x0=[0.01], args=([i]), method="COBYLA")
        optimized_energies[i].append(result.fun)
        print("R={}\t Optimized Energy: {}\t Sampling Noise: {}".format(str(round(diatomic_bond_length, 2)), result.fun, bool(i)))
    print('\n')

R=0.3	 Exact Energy: -0.6018037107656862
R=0.3	 Optimized Energy: -0.5991514719423161	 Sampling Noise: False
R=0.3	 Optimized Energy: -0.6033324162819957	 Sampling Noise: True


R=0.4	 Exact Energy: -0.9141497046270821
R=0.4	 Optimized Energy: -0.9156362870641498	 Sampling Noise: False
R=0.4	 Optimized Energy: -0.9127901548589876	 Sampling Noise: True


R=0.5	 Exact Energy: -1.0551597944706257
R=0.5	 Optimized Energy: -1.0536078022492967	 Sampling Noise: False
R=0.5	 Optimized Energy: -1.0559442162649728	 Sampling Noise: True


R=0.6	 Exact Energy: -1.1162860068695402
R=0.6	 Optimized Energy: -1.1132992653161349	 Sampling Noise: False
R=0.6	 Optimized Energy: -1.116075625172784	 Sampling Noise: True


R=0.7	 Exact Energy: -1.1361894540659225
R=0.7	 Optimized Energy: -1.134581223716484	 Sampling Noise: False
R=0.7	 Optimized Energy: -1.1376068944742828	 Sampling Noise: True


R=0.8	 Exact Energy: -1.1341476666770967
R=0.8	 Optimized Energy: -1.1338783282880063	 Sampling Noise: False
R=0

## Step 7: Plot the dissociation curve

In [10]:
import matplotlib
import matplotlib.pyplot as pyplot

# Plot the various energies for different bond lengths
fig = pyplot.figure(figsize=(10,7))
pyplot.rcParams['font.size']=18
bkcolor = '#ffffff'
ax = fig.add_subplot(1, 1, 1)
pyplot.subplots_adjust(left=.2)
ax.set_xlabel('R (Angstroms)')
ax.set_ylabel(r'E (Hartrees)')
ax.set_title(r'H2 bond dissociation curve')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
bond_lengths = [float(x) for x in bond_lengths]
ax.plot(bond_lengths, optimized_energies[0], 'o', label='UCCSD', color='red')
ax.plot(bond_lengths, optimized_energies[1], 'x', label='UCCSD with Sampling Noise', color='blue')
ax.plot(bond_lengths, exact_energies, '-', label='Full-CI', color='black')

ax.legend(frameon=False)
pyplot.show()